# Create basic component & pipeline

## Here we will create the simplest pipeline possible, only one component that add two inputs

#### Create a simple add function

In [1]:
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a + b)
    return a + b

#### From this function let's create a component

In [17]:
# install dependancies (protobuf, exchange format)
!pip install protobuf==3.20

In [18]:
# import depandancies
import kfp as kfp
import kfp.dsl as dsl
from kfp import components
import os
from kfp.components import InputPath, OutputPath, create_component_from_func #import some functions from the kfp api to help the pipelines creation

In [19]:
# create destiantion folder for components
!mkdir components

mkdir: cannot create directory ‘components’: File exists


In [21]:
### this helper will create a yaml that embed a python image, and some code executed thru it
create_component_from_func(
    add,
    output_component_file='components/add_component.yaml',
    base_image='python:3.7'
)

<function Add(a: float, b: float)>

#### Once this component is created, we will load it into a python variable

In [22]:
add_op = components.load_component_from_file('components/add_component.yaml')

### Create a pipeline using this component

In [23]:
# define metadata for the pipeline
@dsl.pipeline(
    name='addition-pipeline',
    description='An example pipeline that performs addition calculations.',
)
# define the pipeline function, using the add component
def add_pipeline(a: float = 1, b: float = 7):
    add_task = add_op(a, b)

### Create the KFP client to link with Kubeflow Pipeline

In [8]:
### a token has been automatically provided in the KF_PIPELINES_SA_TOKEN_PATH variable. This token allow accès to only your namespace
token_file = os.getenv("KF_PIPELINES_SA_TOKEN_PATH")
with open(token_file) as f:
    token = f.readline()
client = kfp.Client(host='http://ml-pipeline.kubeflow.svc.cluster.local:8888',
               existing_token=token)

Pipeline runs are aggregated by "experiments", you can list it in Kubeflow UI in "experiments(KFP)" menu.

We will create a new experiment, we can use 2 ways to do it
- Within the interface (press new, add a name, validate and then come back to this notebook
- we can create it from the notebook using this snippet, adapted with your namespace name : 



In [9]:
# namespace is made from 
user=''#firstname-lastname
namespace = f'kubeflow-user-{user}'

In [10]:
### create the experiment
EXPERIMENT_NAME = 'Aiengineer labs session2'
experiment = client.create_experiment(name=EXPERIMENT_NAME, namespace=namespace)


In [11]:
# check if your experiments has been created
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2023, 11, 8, 16, 50, 52, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'e8be00fd-127d-4b6f-84ba-946b9a9260c7',
                  'name': 'recette',
                  'resource_references': [{'key': {'id': 'kubeflow-user-guillaume-etevenard',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2023, 11, 10, 11, 4, 30, tzinfo=tzlocal()),
                  'description': None,
                  'id': '26ed83e8-cf86-48ad-81de-c052ae381d77',
                  'name': 'Aiengineer labs session2',
                  'resource_references': [{'key': {'id': 'kubeflow-user-guillaume-etevenard',
                                                   'type': 'NAMES

### Submit the pipeline using the client

In [12]:
import datetime as dt

In [13]:
client.create_run_from_pipeline_func(
    pipeline_func=add_pipeline,
    namespace = namespace,
    experiment_name=EXPERIMENT_NAME,
    run_name=f"add_pipeline_{dt.datetime.today().isoformat()}",
    arguments={}
    )

RunPipelineResult(run_id=c192a0a9-581b-47b1-b388-fff8d31c4266)

### Monitor the result 

You should be able to access your pipeline run, using "experiments" menu of the kubeflow UI, expanding the experiment name you defined just before

![exp](./images/experimentview.png)

If you click on the link, and click on the component in the graph, you can see the result of the "add" open

![results](./images/add_results.png)

### Export a metric to allow better monitoring

In [14]:
def add_with_metric_export(
    a: float, 
    b: float,
    mlpipeline_metrics_path: OutputPath('Metrics')
) -> float:
    '''Calculates sum of two arguments'''
    import json
    
    ### Save and exports metrics ###
    metrics = {
    'metrics': [{
        'name': 'add_result', 
        'numberValue': a + b, 
        }]
    }
    
    with open(mlpipeline_metrics_path, 'w') as f:
        json.dump(metrics, f)

    return a + b

In [15]:
### this helper will create a yaml that embed a python image, and some code executed thru it
create_component_from_func(
    add_with_metric_export,
    output_component_file='components/add_component_with_metric.yaml',
    base_image='python:3.7'
)

<function Add with metric export(a: float, b: float)>

### rerun the pipeline replacing 'add' component with it's new version

In [16]:
add_wm_op = components.load_component_from_file('components/add_component_with_metric.yaml')

# define metadata for the pipeline
@dsl.pipeline(
    name='addition-pipeline',
    description='An other example pipeline that performs addition calculations.',
)
# define the pipeline function, using the add component
def add_wm_pipeline(a: float = 2, b: float = 8):
    add_task = add_wm_op(a, b)
    
client.create_run_from_pipeline_func(
    pipeline_func=add_wm_pipeline,
    namespace = namespace,
    experiment_name=EXPERIMENT_NAME,
    run_name=f"add_pipeline_wm_{dt.datetime.today().isoformat()}",
    arguments={}
    )

RunPipelineResult(run_id=234286a8-fd45-478a-956f-7d67008e5c4f)

Visualize the difference 
![adv](./images/add_result_viz.png)